In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
#from dask.distributed import Client
import dask.dataframe as dd

In [2]:
notes = set() 
input_folder = "kaggle/input/linking-writing-processes-to-writing-quality/"

In [3]:
train_logs = pd.read_csv(input_folder + "train_logs.csv",delimiter = ",",header = 0)
train_scores = pd.read_csv(input_folder +"train_scores.csv", delimiter = ",", header = 0)
scores = pd.Series(data = train_scores['score'].values, index = train_scores['id'].values, name = 'score')

In [4]:
train_logs.head(5)

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [26]:
train_logs.cursor_position.describe()

count    8.405898e+06
mean     1.222964e+03
std      9.485242e+02
min      0.000000e+00
25%      4.990000e+02
50%      1.043000e+03
75%      1.706000e+03
max      7.802000e+03
Name: cursor_position, dtype: float64

In [53]:
#cursor position change var is not significant in the linear model
#action time change var is significant in the linear model
def diff_var(x):
    return np.var(np.diff(np.log(x+1)), ddof = 1)
action_time_change_var = train_logs.groupby('id')['action_time'].agg(diff_var)
action_time_change_var.name = "action_time_change_var"
cursor_position_change_var= train_logs.groupby('id')['cursor_position'].agg(diff_var)
cursor_position_change_var.name = "cursor_position_change_var"
mean_action_time = train_logs.groupby('id')['action_time'].mean()
mean_action_time.name = "mean_action_time"

In [63]:
# Total q over maximum time it takes :
def ncharCounter(x, character):
    n = 0
    for i in x:
        for j in i:
            if j == character:
                n +=1

    return n 

In [84]:
total_q = train_logs.groupby('id')['text_change'].agg(ncharCounter, character = "q")
total_time = train_logs.groupby('id')['action_time'].sum()
efficiency = np.log(total_q / total_time)
efficiency.name = "efficiency"

In [85]:
z = pd.merge(action_time_change_var, cursor_position_change_var, left_index = True, right_index = True)
z = pd.merge(z, efficiency, left_index= True, right_index = True)
z = pd.merge(z,scores, left_index = True, right_index = True)
z.corr()

,action_time_change_var,cursor_position_change_var,efficiency,score
action_time_change_var,1.000000,0.027377,0.011072,-0.093427
cursor_position_change_var,0.027377,1.000000,-0.089600,-0.134337
efficiency,0.011072,-0.089600,1.000000,0.091628
score,-0.093427,-0.134337,0.091628,1.000000


In [52]:
mean_action_time.describe()

count    2471.000000
mean       99.951169
std        24.112678
min         8.423294
25%        83.656015
50%        97.015063
75%       114.520985
max       282.612447
Name: mean_action_time, dtype: float64